# Monitoring Mountain Glacier Melting with SageMaker Geospatial Capabilities


---

This notebook's CI test result for us-west-2 is as follows. CI test results in other regions can be found at the end of the notebook. 

![This us-west-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-west-2/sagemaker-geospatial|mount-shasta-glacier-melting-monitoring|mount_shasta_glacier_melt_monitoring.ipynb)

---



This notebook runs with Kernel Geospatial 1.0. Note that the following policies need to be attached to the execution role that you used to run this notebook:

- AmazonSageMakerFullAccess
- AmazonSageMakerGeospatialFullAccess

You can see the policies attached to the role in the IAM console under the permissions tab. If required, add the roles using the 'Add Permissions' button. 

In addition to these policies, ensure that the execution role's trust policy allows the SageMaker-GeoSpatial service to assume the role. This can be done by adding the following trust policy using the 'Trust relationships' tab:
```
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {
                "Service": [
                    "sagemaker.amazonaws.com",
                    "sagemaker-geospatial.amazonaws.com"
                ]
            },
            "Action": "sts:AssumeRole"
        }
    ]
}
```

**Contents**
* [Setup SageMaker geospatial capabilities](#1)
* [Query Sentinel-2 Data](#2)
* [Start an Earth Observation Job (EOJ) to identify the land cover types in the area of Mount Shasta](#3)
* [Visualize EOJ inputs and outputs in FourSquare Studio](#4)
* [Extract the land cover segmentation results](#5)
* [Measure snow coverage of Mount Shasta](#6)
* [Query Landsat 8 data](#7)
* [Analyze the relationship between snow coverage and surface temperature](#8)



<a id='1'></a>

## Setup SageMaker geospatial capabilities

In [ ]:
import boto3
import sagemaker
import sagemaker_geospatial_map

session = boto3.Session()
execution_role = sagemaker.get_execution_role()
sg_client = session.client(service_name="sagemaker-geospatial")

<a id='2'></a>

## Query Sentinel-2 data

Retrieve Sentinel-2 data over the Mount Shasta area by specifying the data location, time range, and property filters in the query.

In [ ]:
search_rdc_args = {
    "Arn": "arn:aws:sagemaker-geospatial:us-west-2:378778860802:raster-data-collection/public/nmqj48dcu3g7ayw8",  # sentinel-2 L2A COG
    "RasterDataCollectionQuery": {
        "AreaOfInterest": {
            "AreaOfInterestGeometry": {
                "PolygonGeometry": {
                    "Coordinates": [
                        [
                            [-122.198, 41.407],
                            [-122.191, 41.407],
                            [-122.191, 41.411],
                            [-122.198, 41.411],
                            [-122.198, 41.407],
                        ]
                    ]
                }
            }
        },
        "TimeRangeFilter": {
            "StartTime": "2017-07-01T00:00:00Z",
            "EndTime": "2022-07-10T23:59:59Z",
        },
        "PropertyFilters": {
            "Properties": [{"Property": {"EoCloudCover": {"LowerBound": 0, "UpperBound": 1}}}],
            "LogicalOperator": "AND",
        },
        "BandFilter": ["visual"],
    },
}

tci_urls = []
data_manifests = []
while search_rdc_args.get("NextToken", True):
    search_result = sg_client.search_raster_data_collection(**search_rdc_args)
    if search_result.get("NextToken"):
        data_manifests.append(search_result)
    for item in search_result["Items"]:
        tci_url = item["Assets"]["visual"]["Href"]
        # print(tci_url)
        tci_urls.append(tci_url)

    search_rdc_args["NextToken"] = search_result.get("NextToken")

print(f"{len(tci_urls)} images found.")

<a id='3'></a>
## Start an Earth Observation Job (EOJ) to identify the land cover types in the area of Mount Shasta

In [ ]:
# Perform land cover segmentation on images returned from the sentinel dataset.
eoj_input_config = {
    "RasterDataCollectionQuery": {
        "RasterDataCollectionArn": "arn:aws:sagemaker-geospatial:us-west-2:378778860802:raster-data-collection/public/nmqj48dcu3g7ayw8",
        "AreaOfInterest": {
            "AreaOfInterestGeometry": {
                "PolygonGeometry": {
                    "Coordinates": [
                        [
                            [-122.198, 41.407],
                            [-122.191, 41.407],
                            [-122.191, 41.411],
                            [-122.198, 41.411],
                            [-122.198, 41.407],
                        ]
                    ]
                }
            }
        },
        "TimeRangeFilter": {
            "StartTime": "2017-07-01T00:00:00Z",
            "EndTime": "2022-07-10T23:59:59Z",
        },
        "PropertyFilters": {
            "Properties": [{"Property": {"EoCloudCover": {"LowerBound": 0, "UpperBound": 1}}}],
            "LogicalOperator": "AND",
        },
    }
}
eoj_config = {"LandCoverSegmentationConfig": {}}

response = sg_client.start_earth_observation_job(
    Name="mont-shasta-landcover",
    InputConfig=eoj_input_config,
    JobConfig=eoj_config,
    ExecutionRoleArn=execution_role,
)

Monitor the EOJ status

In [ ]:
eoj_arn = response["Arn"]
job_details = sg_client.get_earth_observation_job(Arn=eoj_arn)
{k: v for k, v in job_details.items() if k in ["Arn", "Status", "DurationInSeconds"]}

<a id='4'></a>

## Visualize EOJ inputs and outputs in FourSquare Studio

In [ ]:
# Creates an instance of the map to add EOJ input/ouput layer
map = sagemaker_geospatial_map.create_map({"is_raster": True})
map.set_sagemaker_geospatial_client(sg_client)
# Render the map
map.render()

In [ ]:
# Visualize AOI
config = {"label": "Mount Shasta AOI"}
aoi_layer = map.visualize_eoj_aoi(Arn=eoj_arn, config=config)

In [ ]:
# Visualize input after EOJ has completed.
time_range_filter = {"start_date": "2019-07-01T00:00:00Z", "end_date": "2019-07-10T23:59:59Z"}
config = {"label": "Input"}
input_layer = map.visualize_eoj_input(
    Arn=eoj_arn, config=config, time_range_filter=time_range_filter
)

In [ ]:
# Visualize output, EOJ needs to be in completed status.
time_range_filter = {"start_date": "2019-07-01T00:00:00Z", "end_date": "2019-07-10T23:59:59Z"}
config = {"preset": "singleBand", "band_name": "mask"}
output_layer = map.visualize_eoj_output(
    Arn=eoj_arn, config=config, time_range_filter=time_range_filter
)

<a id='5'></a>
## Extract the land cover segmentation results

In [ ]:
sagemaker_session = sagemaker.Session()
s3_bucket_name = sagemaker_session.default_bucket()  # Replace with your own bucket if needed
s3_bucket = session.resource("s3").Bucket(s3_bucket_name)
prefix = "eoj_montshasta"  # Replace with the S3 prefix desired
export_bucket_and_key = f"s3://{s3_bucket_name}/{prefix}/"

eoj_output_config = {"S3Data": {"S3Uri": export_bucket_and_key}}
export_response = sg_client.export_earth_observation_job(
    Arn=eoj_arn,
    ExecutionRoleArn=execution_role,
    OutputConfig=eoj_output_config,
    ExportSourceImages=False,
)

In [ ]:
# Monitor the export job status
export_job_details = sg_client.get_earth_observation_job(Arn=export_response["Arn"])
{k: v for k, v in export_job_details.items() if k in ["Arn", "ExportStatus"]}

<a id='6'></a>
## Measure snow cover area

In [ ]:
import os
from glob import glob
import cv2
import numpy as np
import tifffile
import matplotlib.pyplot as plt
from urllib.parse import urlparse
from botocore import UNSIGNED
from botocore.config import Config

# Download land cover masks
mask_dir = "./masks/mont_shasta"
os.makedirs(mask_dir, exist_ok=True)
image_paths = []
for s3_object in s3_bucket.objects.filter(Prefix=prefix).all():
    path, filename = os.path.split(s3_object.key)
    if (filename.endswith(".tif")) and (700 < int(filename.split("_")[2][5:8]) < 708):
        mask_name = mask_dir + "/" + filename
        s3_bucket.download_file(s3_object.key, mask_name)
        print("Downloaded mask: " + mask_name)

# Download source images for visualization
image_dir = "./images/mont_shasta"
os.makedirs(image_dir, exist_ok=True)
for tci_url in tci_urls:
    url_parts = urlparse(tci_url)
    img_id = url_parts.path.split("/")[-2]
    # Only use data from the 1st week of July
    if 700 < int(img_id.split("_")[2][5:8]) < 708:
        tci_download_path = image_dir + "/" + img_id + "_TCI.tif"
        cogs_bucket = session.resource(
            "s3", config=Config(signature_version=UNSIGNED, region_name="us-west-2")
        ).Bucket(url_parts.hostname.split(".")[0])
        cogs_bucket.download_file(url_parts.path[1:], tci_download_path)
        print("Downloaded image: " + tci_download_path)

print("Downloads complete.")

Extract the snow mask and measure the snow cover area

In [ ]:
image_files = glob(image_dir + "/*.tif")
mask_files = glob(mask_dir + "/*.tif")
image_files.sort(key=lambda x: x.split("_")[3])
mask_files.sort(key=lambda x: x.split("_")[3])

In [ ]:
overlay_dir = "./masks/mont_shasta_overlay"
os.makedirs(overlay_dir, exist_ok=True)
snow_areas = []
mask_dates = []

for image_file, mask_file in zip(image_files, mask_files):
    image_id = image_file.split("/")[-1].split("_TCI")[0]
    mask_id = mask_file.split("/")[-1].split(".tif")[0]
    mask_date = mask_id.split("_")[2]
    mask_dates.append(mask_date)
    assert image_id == mask_id
    image = tifffile.imread(image_file)
    image_ds = cv2.resize(image, (1830, 1830), interpolation=cv2.INTER_LINEAR)
    snow_mount_img = image_ds[100:400, 1000:1250]  # a crop of the Mount Shasta area
    mask = tifffile.imread(mask_file)
    snow_mask = np.isin(mask, [11]).astype(np.uint8)  # extract snow mask
    snow_mount_mask = snow_mask[100:400, 1000:1250]
    snow_area = snow_mount_mask.sum() * 60 * 60 / (1000 * 1000)  # calculate the snow cover area
    snow_areas.append(snow_area)
    red_img = np.zeros(snow_mount_img.shape, snow_mount_img.dtype)
    red_img[:, :] = (255, 0, 0)
    red_mask = cv2.bitwise_and(red_img, red_img, mask=snow_mount_mask)
    overlay_img = cv2.addWeighted(red_mask, 1, snow_mount_img, 1, 0, snow_mount_img)
    cv2.putText(
        overlay_img,
        f"{mask_date}",
        (0, 20),
        cv2.FONT_HERSHEY_SIMPLEX,
        0.5,
        (255, 0, 0),
        1,
        cv2.LINE_AA,
    )
    cv2.putText(
        overlay_img,
        f"{snow_area} [sq km]",
        (0, 40),
        cv2.FONT_HERSHEY_SIMPLEX,
        0.5,
        (255, 0, 0),
        1,
        cv2.LINE_AA,
    )
    overlay_file = overlay_dir + "/" + mask_date + ".png"
    cv2.imwrite(overlay_file, cv2.cvtColor(overlay_img, cv2.COLOR_RGB2BGR))

In [ ]:
# Average snow area in July
snow_area_date = {}
for snow_area, mask_date in zip(snow_areas, mask_dates):
    mask_date = mask_date[:6]  # year-month
    if mask_date in snow_area_date:
        snow_area_date[mask_date] += np.array([snow_area, 1])
    else:
        snow_area_date[mask_date] = np.array([snow_area, 1])
dates = list(snow_area_date.keys())
snow_area_year = np.zeros(len(dates))
for i, snow_area in enumerate(snow_area_date.values()):
    snow_area_year[i] = snow_area[0] / snow_area[1]

# Plot snow cover area vs. time.
plt.figure(figsize=(10, 5))
plt.title("Mount Shasta snow area from 2017 to 2022.", fontsize=20)
plt.xticks(rotation=45)
plt.ylabel("Snow cover area [sq km]", fontsize=14)
plt.plot(dates, snow_area_year, marker="o")
for i, v in enumerate(snow_area_year):
    plt.text(i, v + 1.5, "%d" % v, ha="center")
plt.show()

In [ ]:
import imageio.v2 as imageio

frames = []
filenames = glob(overlay_dir + "/*.png")
filenames.sort()

for filename in filenames:
    frames.append(imageio.imread(filename))
imageio.mimsave("mont_shasta.gif", frames, duration=2)

In [ ]:
from IPython.display import HTML

HTML('<img src="./mont_shasta.gif" width="400">')

<a id='7'></a>

## Query Landsat 8 data
Retrieve the Landsat surface temperature data to gain a better understanding of the changes that have taken place in the same area over the past 5 years.

In [ ]:
search_rdc_args = {
    "Arn": "arn:aws:sagemaker-geospatial:us-west-2:378778860802:raster-data-collection/public/gmqa64dcu2g9ayx1",  # Landsat 8 C2L2
    "RasterDataCollectionQuery": {
        "AreaOfInterest": {
            "AreaOfInterestGeometry": {
                "PolygonGeometry": {
                    "Coordinates": [
                        [
                            [-122.198, 41.407],
                            [-122.191, 41.407],
                            [-122.191, 41.411],
                            [-122.198, 41.411],
                            [-122.198, 41.407],
                        ]
                    ]
                }
            }
        },
        "TimeRangeFilter": {
            "StartTime": "2017-07-01T00:00:00Z",
            "EndTime": "2022-07-30T23:59:59Z",
        },
        "PropertyFilters": {
            "Properties": [{"Property": {"EoCloudCover": {"LowerBound": 0, "UpperBound": 1}}}],
            "LogicalOperator": "AND",
        },
        "BandFilter": ["lwir11"],
    },
}

lst_urls = []
data_manifests = []
while search_rdc_args.get("NextToken", True):
    search_result = sg_client.search_raster_data_collection(**search_rdc_args)
    if search_result.get("NextToken"):
        data_manifests.append(search_result)
    for item in search_result["Items"]:
        lst_url = item["Assets"]["lwir11"]["Href"]
        print(lst_url)
        lst_urls.append(lst_url)

    search_rdc_args["NextToken"] = search_result.get("NextToken")

Download Landsat surface temperature data

In [ ]:
s3 = boto3.client("s3")
lst_dir = "./images/mont_shasta_lst"
os.makedirs(lst_dir, exist_ok=True)
lst_paths = []

for lst_url in lst_urls:
    url_parts = urlparse(lst_url)
    img_id = url_parts.path.split("/")[-2]
    if 700 < int(img_id[22:25]) < 731:
        lst_download_path = lst_dir + "/" + img_id + "_ST.tif"
        s3.download_file(
            url_parts.netloc,
            url_parts.path[1:],
            lst_download_path,
            ExtraArgs={"RequestPayer": "requester"},
        )
        print("Downloaded LST image: " + img_id)

In [ ]:
lst_files = glob(lst_dir + "/*.tif")
lst_files.sort(key=lambda x: x.split("_")[5])

Landsat surface temperature: https://www.usgs.gov/faqs/how-do-i-use-scale-factor-landsat-level-2-science-products


In [ ]:
mont_lsts = []
lst_dates = []

for lst_file in lst_files:
    lst_date = lst_file.split("_")[5]
    lst_dates.append(lst_date)
    lst = tifffile.imread(lst_file)
    mont_lst = lst[5000:5500, 3900:4400]  # bounding box for Mount Shasta
    mont_lst_c = (np.mean(mont_lst) * 0.00341802 + 149) - 273  # surface temperature in Celsius
    mont_lsts.append(mont_lst_c)

<a id='8'></a>

## Relationship between surface temperature and snow cover area

In [ ]:
# Average surface temperature in July
mont_temp_date = {}
for mont_lst, lst_date in zip(mont_lsts, lst_dates):
    lst_date = lst_date[:6]  # year-month
    if lst_date in mont_temp_date:
        mont_temp_date[lst_date] += np.array([mont_lst, 1])
    else:
        mont_temp_date[lst_date] = np.array([mont_lst, 1])
dates = list(mont_temp_date.keys())
mont_temp_year = np.zeros(len(dates))
for i, mont_temp in enumerate(mont_temp_date.values()):
    mont_temp_year[i] = mont_temp[0] / mont_temp[1]

In [ ]:
from scipy.stats import pearsonr

# Pearson's r to measure the strength and direction of the relationship between two variables
res = pearsonr(snow_area_year, mont_temp_year)
print("Pearson's correlction coefficient: %.2f" % res.statistic)

In [ ]:
fig, ax1 = plt.subplots(figsize=(8, 4))
ax2 = ax1.twinx()
lns1 = ax1.plot(dates, snow_area_year, "b-", marker="o", label="Snow cover area")
for i, v in enumerate(snow_area_year):
    ax1.text(i, v - 5, "%d" % v, ha="center")
lns2 = ax2.plot(dates, mont_temp_year, "r-", marker="x", label="Surface temperature")
for i, v in enumerate(mont_temp_year):
    ax2.text(i, v + 0.2, "%d" % v, ha="center")
ax1.set_ylabel("Snow cover area [sq km]", fontsize=12)
ax2.set_ylabel("Surface temperature [\N{DEGREE SIGN}C]", fontsize=12)
lns = lns1 + lns2
labs = [l.get_label() for l in lns]
ax1.legend(lns, labs, loc=7)
plt.title("Pearsons correlation: %.2f" % res.statistic)
plt.show()

## Notebook CI Test Results

This notebook was tested in multiple regions. The test results are as follows, except for us-west-2 which is shown at the top of the notebook.

![This us-east-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-east-1/sagemaker-geospatial|mount-shasta-glacier-melting-monitoring|mount_shasta_glacier_melt_monitoring.ipynb)

![This us-east-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-east-2/sagemaker-geospatial|mount-shasta-glacier-melting-monitoring|mount_shasta_glacier_melt_monitoring.ipynb)

![This us-west-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-west-1/sagemaker-geospatial|mount-shasta-glacier-melting-monitoring|mount_shasta_glacier_melt_monitoring.ipynb)

![This ca-central-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ca-central-1/sagemaker-geospatial|mount-shasta-glacier-melting-monitoring|mount_shasta_glacier_melt_monitoring.ipynb)

![This sa-east-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/sa-east-1/sagemaker-geospatial|mount-shasta-glacier-melting-monitoring|mount_shasta_glacier_melt_monitoring.ipynb)

![This eu-west-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-west-1/sagemaker-geospatial|mount-shasta-glacier-melting-monitoring|mount_shasta_glacier_melt_monitoring.ipynb)

![This eu-west-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-west-2/sagemaker-geospatial|mount-shasta-glacier-melting-monitoring|mount_shasta_glacier_melt_monitoring.ipynb)

![This eu-west-3 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-west-3/sagemaker-geospatial|mount-shasta-glacier-melting-monitoring|mount_shasta_glacier_melt_monitoring.ipynb)

![This eu-central-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-central-1/sagemaker-geospatial|mount-shasta-glacier-melting-monitoring|mount_shasta_glacier_melt_monitoring.ipynb)

![This eu-north-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-north-1/sagemaker-geospatial|mount-shasta-glacier-melting-monitoring|mount_shasta_glacier_melt_monitoring.ipynb)

![This ap-southeast-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-southeast-1/sagemaker-geospatial|mount-shasta-glacier-melting-monitoring|mount_shasta_glacier_melt_monitoring.ipynb)

![This ap-southeast-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-southeast-2/sagemaker-geospatial|mount-shasta-glacier-melting-monitoring|mount_shasta_glacier_melt_monitoring.ipynb)

![This ap-northeast-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-northeast-1/sagemaker-geospatial|mount-shasta-glacier-melting-monitoring|mount_shasta_glacier_melt_monitoring.ipynb)

![This ap-northeast-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-northeast-2/sagemaker-geospatial|mount-shasta-glacier-melting-monitoring|mount_shasta_glacier_melt_monitoring.ipynb)

![This ap-south-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-south-1/sagemaker-geospatial|mount-shasta-glacier-melting-monitoring|mount_shasta_glacier_melt_monitoring.ipynb)
